In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import pymongo
import cx_Oracle


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [2]:
conn_str = u'report/justd0it@10.234.152.61:1521/alfabi'
connection = cx_Oracle.connect(conn_str)

conn_mongo = pymongo.MongoClient\
        ("mongodb://127.0.0.1:27017/seq_trans")
mydb = conn_mongo["seq_trans"]
print(mydb.list_collection_names())

mycol = mydb['member_item_seq']

['member_item_seq']


In [3]:
q_test = """
SELECT TR_MEMBERS, TR_PLU_PCS, TR_DATE, TR_QTY, LOG_PROMO
FROM RPT_TRANS rt 
WHERE tr_date between '11nov20' AND '11nov20'
AND dw_branch_code = 'TZ01'
AND TR_MEMBERS != 'N'
and NVL(rt.tr_members,'N') not like '%#%'
and NVL(rt.tr_members,'N') not in (select mcp_no_kartu from report.rpt_ms_ssp)
and NVL(rt.tr_type,'N') not like '%R%'
and NVL(rt.tr_type,'-') not like '%X%'
and NVL(rt.tr_type,'-') not like '%V%'
and NVL(rt.tr_flag_rpt,'N') not in ('S','B','X')
and NVL(rt.tr_type, '-') not in ('S', 'B')
and length(rt.tr_members) in (10,12,16)
"""


df_trans_hist = pd.read_sql_query(q_test, connection)   

In [4]:
df_trans_hist.head()

,TR_MEMBERS,TR_PLU_PCS,TR_DATE,TR_QTY,LOG_PROMO
0,9990010552953647,103001,2020-11-11,1.0,None
1,9990010552953647,156210,2020-11-11,1.0,T-J20-4422|5900
2,9990010552953647,164338,2020-11-11,1.0,None
3,9990010552953647,401181,2020-11-11,1.0,None
4,9990010552953647,403990,2020-11-11,2.0,None


In [5]:
df_trans_hist['TR_DATE'] = df_trans_hist.TR_DATE.values.astype(np.int64) // 10 ** 9

df_trans_hist['LOG_PROMO'] = df_trans_hist['LOG_PROMO'].fillna('-')
df_trans_hist = df_trans_hist.groupby(['TR_MEMBERS','TR_PLU_PCS','TR_DATE'])\
                    .agg({'TR_QTY':'sum','LOG_PROMO':' '.join}).reset_index()
df_trans_hist['TR_QTY'] = df_trans_hist['TR_QTY'].astype('int')
# df_trans_hist['LOG_PROMO_BOOL'] = df_trans_hist['LOG_PROMO']\
#                     .apply(lambda x: 'N' if x.startswith('-') else 'P')

df_trans_hist['tr_date_count'] = list(zip(df_trans_hist.TR_QTY, df_trans_hist.LOG_PROMO))

df_trans_hist['$set'] = pd.Series([{'TRF_DATE.{}'.format(df_trans_hist['TR_DATE'][i]):(df_trans_hist['tr_date_count'][i])}\
                          for i in range(len(df_trans_hist))])

In [6]:
# df_trans_hist['set_date'] = 'TRF_DATE.' + df_trans_hist['TR_DATE'].astype(str)

In [7]:
# df_trans_hist['$set_test'] = pd.Series(df_trans_hist.tr_date_count.values,index=df_trans_hist.set_date).to_dict()

In [8]:
df_trans_hist.head()

,TR_MEMBERS,TR_PLU_PCS,TR_DATE,TR_QTY,LOG_PROMO,tr_date_count,$set
0,000105148434,101643,1605052800,1,T-J20-2806|700,"(1, T-J20-2806|700)","{'TRF_DATE.1605052800': (1, 'T-J20-2806|700')}"
1,000105148434,103001,1605052800,1,-,"(1, -)","{'TRF_DATE.1605052800': (1, '-')}"
2,000105148434,110127,1605052800,1,T-K20-1442|8200,"(1, T-K20-1442|8200)","{'TRF_DATE.1605052800': (1, 'T-K20-1442|8200')}"
3,000105148434,121637,1605052800,2,T-J20-3268|1100,"(2, T-J20-3268|1100)","{'TRF_DATE.1605052800': (2, 'T-J20-3268|1100')}"
4,000105148434,320383,1605052800,1,-,"(1, -)","{'TRF_DATE.1605052800': (1, '-')}"
